In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
import math

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
torch.cuda.is_available() 

True

# Dataloader

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Custom Dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data.float()

    def __getitem__(self, index):
        return self.data[index].float()

    def __len__(self):
        return len(self.data)

In [5]:
def train_epoch(train_loader, model, optimizer, loss_function):
    losses = []
    # get a batch of training data from the train_loader (DataLoader obj)
    for i, data in enumerate(train_loader):
        x = data[:,:-1].to(device)
        t = data[:,-1].unsqueeze(1).to(device)
        # make predictions for this batch
        y = model(x)
        # Compute the loss
        loss = loss_function(y, t)
        # zero out the gradients so that it will not accumulate through each iteration
        optimizer.zero_grad()
        # Compute the gradents with the backward call (backprop)
        loss.backward()
        # Update weight using gradient descent 
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_epoch(valid_loader, model, loss_function):
    rmses = []
    with torch.no_grad(): 
        for i, data in enumerate(valid_loader):
            x = data[:,:-1].to(device)
            t = data[:,-1].unsqueeze(1).to(device)
            # Compute prediction
            y = model(x)
            
            rmse = math.sqrt(loss_function(y,t))
            rmses.append(rmse)
            
    return np.mean(rmses)

# Define Model Architectures

In [42]:
# Define the neural network architecture
leaky_relu = nn.LeakyReLU(negative_slope=0.2)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.selu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(104, 64)
        # self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32,16)
        self.fc5 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.selu(self.fc1(x))
        x = self.dropout(x)
        # x = self.dropout(self.relu(self.fc2(x)))
        x = self.selu(self.fc3(x))
        x = self.dropout(x)
        x = self.selu(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)
        return x

# Create an instance of the neural network
model = Net()
model = model.to(device)
criterion = nn.MSELoss()

# Load in Data

In [43]:
df_np = np.load('data/processed_train_set.npy')
df_np.shape

(1710670, 105)

In [44]:
# load in np array and DRAW SAMPLE
num_samples = int(df_np.shape[0]*1)

num_rows = df_np.shape[0]
sampled_indices = np.random.choice(num_rows, size=num_samples, replace=False)

In [45]:
# convert np to tensor
df = torch.tensor(df_np[sampled_indices])

In [46]:
df.shape

torch.Size([1710670, 105])

In [47]:
# Create a sample dataset
dataset = MyDataset(df)

# Define the sizes of train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset)-train_size

In [48]:
# Split the dataset
batch_size = 256
train_data, val_data = random_split(dataset, [train_size, val_size])

# Create DataLoader objects for train, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=batch_size)
# test_loader = DataLoader(test_data, batch_size=32)

# Train Model

In [49]:
def train_model(model, model_name, epochs, lr, batch_size):
    # Define the optimizer
    train_rmses = []
    valid_rmses = []
    optimizer = torch.optim.Adam(model.parameters(), lr)
#     optimizer = torch.optim.Adam(model.parameters())
    train_accs, valid_accs = [], []
    for epoch in range(epochs):

        model.train() # gradient tracking is on

        train_loss = train_epoch(train_loader, model, optimizer, criterion)

        model.eval() # we don't need gradients on to do reporting

        train_rmse = eval_epoch(train_loader, model, criterion)
        valid_rmse = eval_epoch(valid_loader, model, criterion)
        
#         train_rmses.append(train_rmse)
#         valid_rmses.append(valid_rmse)

        print(f"Epoch: {epoch+1}, Train Loss: {train_loss:>0.4f}, Train mse: {train_rmse:>0.2f},\
              Validation Accuracy: {valid_rmse:>0.2f} \n")
    torch.save(model.state_dict(), './{}'.format(model_name))


In [52]:
%%time
train_model(model = model, model_name = 'main_model', \
epochs = 20, lr=0.001, batch_size = batch_size)

Epoch: 1, Train Loss: 513487.6387, Train mse: 624.18,              Validation Accuracy: 618.92 

Epoch: 2, Train Loss: 509012.0057, Train mse: 624.34,              Validation Accuracy: 619.84 

Epoch: 3, Train Loss: 504232.2777, Train mse: 621.78,              Validation Accuracy: 616.41 

Epoch: 4, Train Loss: 501043.2747, Train mse: 620.87,              Validation Accuracy: 615.68 

Epoch: 5, Train Loss: 498434.6600, Train mse: 620.67,              Validation Accuracy: 615.70 

Epoch: 6, Train Loss: 496418.1310, Train mse: 620.03,              Validation Accuracy: 615.62 

Epoch: 7, Train Loss: 494318.9274, Train mse: 619.09,              Validation Accuracy: 614.08 

Epoch: 8, Train Loss: 492711.7006, Train mse: 621.03,              Validation Accuracy: 616.38 

Epoch: 9, Train Loss: 491548.9129, Train mse: 620.03,              Validation Accuracy: 614.86 

Epoch: 10, Train Loss: 490487.9378, Train mse: 618.25,              Validation Accuracy: 613.90 

Epoch: 11, Train Loss: 489508

In [51]:
# saved_model = Net()
# saved_model.load_state_dict(torch.load('./cse151B_project'))
# saved_model

# Test Model

In [53]:
df_np_test = np.load('data/processed_test_set.npy')
df_np_test.shape

(320, 104)

In [54]:
test_tensor = torch.tensor(df_np_test).to(device)
test_tensor.shape

torch.Size([320, 104])

In [55]:
model = model.float()
model

Net(
  (selu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=104, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=1, bias=True)
)

In [56]:
pred = []
with torch.no_grad(): 
    for i in range(test_tensor.shape[0]):
        x = test_tensor[i].float().to(device)
        y = model(x)
        pred.append(y.cpu().numpy()[0])         

In [57]:
pred

[743.87317,
 653.98926,
 743.87317,
 664.11597,
 677.2739,
 693.7875,
 743.87317,
 693.7875,
 732.06445,
 743.87317,
 603.3226,
 768.93933,
 768.93933,
 768.93933,
 768.93933,
 768.93933,
 768.93933,
 768.93933,
 743.87317,
 768.93933,
 743.87317,
 693.7875,
 765.0043,
 743.87317,
 635.8312,
 631.9723,
 743.87317,
 743.87317,
 631.9232,
 743.87317,
 743.87317,
 653.98926,
 768.93933,
 768.93933,
 768.93933,
 768.93933,
 813.77075,
 616.83325,
 743.87317,
 665.41895,
 649.6824,
 768.93933,
 768.93933,
 732.06445,
 631.9723,
 646.7499,
 768.93933,
 768.93933,
 542.9862,
 603.3226,
 604.21,
 633.1794,
 616.83325,
 743.87317,
 743.87317,
 617.0306,
 732.06445,
 617.0105,
 768.93933,
 693.7875,
 743.87317,
 768.93933,
 768.93933,
 813.77075,
 732.06445,
 732.06445,
 743.87317,
 693.7875,
 768.93933,
 604.21,
 814.95374,
 768.93933,
 743.87317,
 768.93933,
 615.1007,
 933.3453,
 612.88086,
 933.3453,
 933.3453,
 933.3453,
 700.7935,
 615.1007,
 700.7935,
 700.7935,
 628.9321,
 700.7935,
 933

In [58]:
test_df = pd.read_csv('data/test_public.csv')
test_data = {'TRIP_ID': test_df['TRIP_ID'],
    'TRAVEL_TIME': pred}
output = pd.DataFrame(data = test_data)
output

,TRIP_ID,TRAVEL_TIME
0,T1,743.873169
1,T2,653.989258
2,T3,743.873169
3,T4,664.115967
4,T5,677.273926
...,...,...
315,T323,634.837280
316,T324,628.094849
317,T325,718.346008
318,T326,634.837280


In [60]:
output.to_csv('data/nathan_test_june5_3_2.csv', index=False)
